<a href="https://colab.research.google.com/github/hheidy0463/fma/blob/main/songrecknn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl -O https://os.unil.cloud.switch.ch/fma/fma_small.zip
!unzip fma_small.zip
!rm fma_small.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7323M  100 7323M    0     0  20.6M      0  0:05:54  0:05:54 --:--:-- 21.4M
Archive:  fma_small.zip
replace fma_small/README.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
pwd

'/content'

In [ ]:
ls /content

fma_metadata/  fma_small/  sample_data/


In [ ]:
import os
os.environ['AUDIO_DIR'] = '/content/fma_small'

In [ ]:
# ensure some files are accessible

audio_dir = os.getenv('AUDIO_DIR')
print("Directory content:", os.listdir(audio_dir)[:10])

Directory content: ['050', '034', '117', '112', '145', '073', '046', '030', '070', '054']


In [ ]:
ls fma_small/

000/  011/  022/  033/  044/  055/  066/  077/  088/  099/  110/  121/  132/  143/  154/
001/  012/  023/  034/  045/  056/  067/  078/  089/  100/  111/  122/  133/  144/  155/
002/  013/  024/  035/  046/  057/  068/  079/  090/  101/  112/  123/  134/  145/  checksums
003/  014/  025/  036/  047/  058/  069/  080/  091/  102/  113/  124/  135/  146/  README.txt
004/  015/  026/  037/  048/  059/  070/  081/  092/  103/  114/  125/  136/  147/
005/  016/  027/  038/  049/  060/  071/  082/  093/  104/  115/  126/  137/  148/
006/  017/  028/  039/  050/  061/  072/  083/  094/  105/  116/  127/  138/  149/
007/  018/  029/  040/  051/  062/  073/  084/  095/  106/  117/  128/  139/  150/
008/  019/  030/  041/  052/  063/  074/  085/  096/  107/  118/  129/  140/  151/
009/  020/  031/  042/  053/  064/  075/  086/  097/  108/  119/  130/  141/  152/
010/  021/  032/  043/  054/  065/  076/  087/  098/  109/  120/  131/  142/  153/


In [ ]:
some_sub_dir = os.path.join(audio_dir, '002')
print("Sub-directory content:", os.listdir(some_sub_dir)[:10])

Sub-directory content: ['002097.mp3', '002012.mp3', '002099.mp3', '002096.mp3']


In [ ]:
# Download file and unzip
!curl -O https://os.unil.cloud.switch.ch/fma/fma_metadata.zip
!unzip fma_metadata.zip
!rm fma_metadata.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  341M  100  341M    0     0  18.4M      0  0:00:18  0:00:18 --:--:-- 20.6M
Archive:  fma_metadata.zip
 bunzipping: fma_metadata/README.txt  
 bunzipping: fma_metadata/checksums  
 bunzipping: fma_metadata/not_found.pickle  
 bunzipping: fma_metadata/raw_genres.csv  
 bunzipping: fma_metadata/raw_albums.csv  
 bunzipping: fma_metadata/raw_artists.csv  
 bunzipping: fma_metadata/raw_tracks.csv  
 bunzipping: fma_metadata/tracks.csv  
 bunzipping: fma_metadata/genres.csv  
 bunzipping: fma_metadata/raw_echonest.csv  
 bunzipping: fma_metadata/echonest.csv  
 bunzipping: fma_metadata/features.csv  


In [ ]:
import os
print("Current Working Directory:", os.getcwd())


Current Working Directory: /content


In [ ]:
print("Directory Contents:", os.listdir())


Directory Contents: ['.config', 'fma_small', 'fma_metadata', 'sample_data']


In [ ]:
import zipfile
import os

# Define the path to the zip file and the extraction directory
zip_file_path = '/content/fma_metadata'
extraction_directory = os.getcwd()  # or any directory where you want to extract files

if os.path.exists(zip_file_path):
    print(f"ZIP file exists at {zip_file_path}")
else:
    print("ZIP file does not exist.")

ZIP file exists at /content/fma_metadata


In [ ]:
import os
import multiprocessing
import warnings
import numpy as np
from scipy import stats
import pandas as pd
import librosa
from tqdm import tqdm
import ast  # Make sure this line is added


In [ ]:
# Define the path where you expect the 'tracks.csv' file
tracks_csv_path = '/content/fma_metadata/tracks.csv'

# Check if tracks.csv exists at the specified path
if os.path.exists(tracks_csv_path):
    print(f"tracks.csv exists at {tracks_csv_path}")
else:
    print("tracks.csv does not exist at the expected location.")


tracks.csv exists at /content/fma_metadata/tracks.csv


In [ ]:
# Number of samples per 30s audio clip.
# TODO: fix dataset to be constant.
NB_AUDIO_SAMPLES = 1321967
SAMPLING_RATE = 44100
import numpy as np

class Loader:
    def load(self, filepath):
        raise NotImplementedError()

class RawAudioLoader(Loader):
    def __init__(self, sampling_rate=SAMPLING_RATE):
        self.sampling_rate = sampling_rate
        self.nb_samples = NB_AUDIO_SAMPLES

    def load(self, filepath):
        data = self._load(filepath)
        # Pad the waveform if it's shorter than expected
        if len(data) < self.nb_samples:
            padding = np.zeros(self.nb_samples - len(data))
            data = np.concatenate((data, padding))
        # Trim the waveform if it's longer than expected
        elif len(data) > self.nb_samples:
            data = data[:self.nb_samples]
        return data

class LibrosaLoader(RawAudioLoader):
    def _load(self, filepath):
        import librosa
        # Use 'None' for original sampling rate, or set a new rate
        sr = self.sampling_rate if self.sampling_rate != SAMPLING_RATE else None
        # Load with librosa
        x, _ = librosa.load(filepath, sr=sr, res_type='kaiser_fast')
        return x


In [ ]:
#!/usr/bin/env python3

# FMA: A Dataset For Music Analysis
# Michaël Defferrard, Kirell Benzi, Pierre Vandergheynst, Xavier Bresson, EPFL LTS2.

# All features are extracted using [librosa](https://github.com/librosa/librosa).
# Alternatives:
# * [Essentia](http://essentia.upf.edu) (C++ with Python bindings)
# * [MARSYAS](https://github.com/marsyas/marsyas) (C++ with Python bindings)
# * [RP extract](http://www.ifs.tuwien.ac.at/mir/downloads.html) (Matlab, Java, Python)
# * [jMIR jAudio](http://jmir.sourceforge.net) (Java)
# * [MIRtoolbox](https://www.jyu.fi/hum/laitokset/musiikki/en/research/coe/materials/mirtoolbox) (Matlab)

import os
import multiprocessing
import warnings
import numpy as np
from scipy import stats
import pandas as pd
import librosa
from tqdm import tqdm

#utils
def load(filepath):

    filename = os.path.basename(filepath)

    if 'features' in filename:
        return pd.read_csv(filepath, index_col=0, header=[0, 1, 2])

    if 'echonest' in filename:
        return pd.read_csv(filepath, index_col=0, header=[0, 1, 2])

    if 'genres' in filename:
        return pd.read_csv(filepath, index_col=0)

    if 'tracks' in filename:
        tracks = pd.read_csv(filepath, index_col=0, header=[0, 1])

        COLUMNS = [('track', 'tags'), ('album', 'tags'), ('artist', 'tags'),
                   ('track', 'genres'), ('track', 'genres_all')]
        for column in COLUMNS:
            tracks[column] = tracks[column].map(ast.literal_eval)

        COLUMNS = [('track', 'date_created'), ('track', 'date_recorded'),
                   ('album', 'date_created'), ('album', 'date_released'),
                   ('artist', 'date_created'), ('artist', 'active_year_begin'),
                   ('artist', 'active_year_end')]
        for column in COLUMNS:
            tracks[column] = pd.to_datetime(tracks[column])

        SUBSETS = ('small', 'medium', 'large')
        try:
            tracks['set', 'subset'] = tracks['set', 'subset'].astype(
                    'category', categories=SUBSETS, ordered=True)
        except (ValueError, TypeError):
            # the categories and ordered arguments were removed in pandas 0.25
            tracks['set', 'subset'] = tracks['set', 'subset'].astype(
                     pd.CategoricalDtype(categories=SUBSETS, ordered=True))

        COLUMNS = [('track', 'genre_top'), ('track', 'license'),
                   ('album', 'type'), ('album', 'information'),
                   ('artist', 'bio')]
        for column in COLUMNS:
            tracks[column] = tracks[column].astype('category')

        return tracks

def columns():
    feature_sizes = dict(chroma_stft=12, chroma_cqt=12, chroma_cens=12,
                         tonnetz=6, mfcc=20, rmse=1, zcr=1,
                         spectral_centroid=1, spectral_bandwidth=1,
                         spectral_contrast=7, spectral_rolloff=1)
    moments = ('mean', 'std', 'skew', 'kurtosis', 'median', 'min', 'max')

    columns = []
    for name, size in feature_sizes.items():
        for moment in moments:
            it = ((name, moment, '{:02d}'.format(i+1)) for i in range(size))
            columns.extend(it)

    names = ('feature', 'statistics', 'number')
    columns = pd.MultiIndex.from_tuples(columns, names=names)

    # More efficient to slice if indexes are sorted.
    return columns.sort_values()


def compute_features(tid):

    features = pd.Series(index=columns(), dtype=np.float32, name=tid)
    loader = LibrosaLoader(sampling_rate=44100)  # Initialize your custom loader


    def feature_stats(name, values):
        features[(name, 'mean')] = np.mean(values, axis=1)
        features[(name, 'std')] = np.std(values, axis=1)
        features[(name, 'skew')] = stats.skew(values, axis=1)
        features[(name, 'kurtosis')] = stats.kurtosis(values, axis=1)
        features[(name, 'median')] = np.median(values, axis=1)
        features[(name, 'min')] = np.min(values, axis=1)
        features[(name, 'max')] = np.max(values, axis=1)


    try:
        filepath = get_audio_path(os.environ.get('AUDIO_DIR'), tid)
        x = loader.load(filepath)  # Load audio data using the custom loader
        assert len(x) == NB_AUDIO_SAMPLES, f"Track {tid} length mismatch: {len(x)} samples, expected {NB_AUDIO_SAMPLES}"

        sr = loader.sampling_rate  # Ensure sampling rate consistency

        f = librosa.feature.zero_crossing_rate(x, frame_length=2048, hop_length=512)
        feature_stats('zcr', f)

        cqt = np.abs(librosa.cqt(x, sr=sr, hop_length=512, bins_per_octave=12,
                                 n_bins=7*12, tuning=None))
        assert cqt.shape[0] == 7 * 12
        assert np.ceil(len(x)/512) <= cqt.shape[1] <= np.ceil(len(x)/512)+1

        f = librosa.feature.chroma_cqt(C=cqt, n_chroma=12, n_octaves=7)
        feature_stats('chroma_cqt', f)
        f = librosa.feature.chroma_cens(C=cqt, n_chroma=12, n_octaves=7)
        feature_stats('chroma_cens', f)
        f = librosa.feature.tonnetz(chroma=f)
        feature_stats('tonnetz', f)

        del cqt
        stft = np.abs(librosa.stft(x, n_fft=2048, hop_length=512))
        assert stft.shape[0] == 1 + 2048 // 2
        assert np.ceil(len(x)/512) <= stft.shape[1] <= np.ceil(len(x)/512)+1
        del x

        f = librosa.feature.chroma_stft(S=stft**2, n_chroma=12)
        feature_stats('chroma_stft', f)

        f = librosa.feature.rmse(S=stft)
        feature_stats('rmse', f)

        f = librosa.feature.spectral_centroid(S=stft)
        feature_stats('spectral_centroid', f)
        f = librosa.feature.spectral_bandwidth(S=stft)
        feature_stats('spectral_bandwidth', f)
        f = librosa.feature.spectral_contrast(S=stft, n_bands=6)
        feature_stats('spectral_contrast', f)
        f = librosa.feature.spectral_rolloff(S=stft)
        feature_stats('spectral_rolloff', f)

        mel = librosa.feature.melspectrogram(sr=sr, S=stft**2)
        del stft
        f = librosa.feature.mfcc(S=librosa.power_to_db(mel), n_mfcc=20)
        feature_stats('mfcc', f)

    except Exception as e:
        print('{}: {}'.format(tid, repr(e)))

    return features


def main():
    tracks = load('tracks.csv')
    specific_tids = [2, 3, 5, 10, 20, 26, 30, 46]

    features = pd.DataFrame(index=tracks.index(specefic_tids, name='track_id'),
                            columns=columns(), dtype=np.float32)

    # More than usable CPUs to be CPU bound, not I/O bound. Beware memory.
    nb_workers = min(4, len(os.sched_getaffinity(0)))
    # int(1.5 * len(os.sched_getaffinity(0)))

    pool = multiprocessing.Pool(nb_workers)
    it = pool.imap_unordered(compute_features, specific_tids)

    for i, row in enumerate(tqdm(it, total=len(specific_tids))):
        features.loc[row.name] = row

        if i % 2 == 0:
            save(features, 10)

    save(features, 10)
    test(features, 10)


def save(features, ndigits):

    # Should be done already, just to be sure.
    features.sort_index(axis=0, inplace=True)
    features.sort_index(axis=1, inplace=True)

    features.to_csv('features.csv', float_format='%.{}e'.format(ndigits))


def test(features, ndigits):

    indices = features[features.isnull().any(axis=1)].index
    if len(indices) > 0:
        print('Failed tracks: {}'.format(', '.join(str(i) for i in indices)))

    tmp = load('features.csv')
    np.testing.assert_allclose(tmp.values, features.values, rtol=10**-ndigits)



In [ ]:
# Assuming tracks.csv was found in the previous step
tracks = load(tracks_csv_path)


In [ ]:
# Assuming tracks.csv was found and loaded successfully
print(tracks)


            album                                                              \
         comments        date_created date_released engineer favorites     id   
track_id                                                                        
2               0 2008-11-26 01:44:45    2009-01-05      NaN         4      1   
3               0 2008-11-26 01:44:45    2009-01-05      NaN         4      1   
5               0 2008-11-26 01:44:45    2009-01-05      NaN         4      1   
10              0 2008-11-26 01:45:08    2008-02-06      NaN         4      6   
20              0 2008-11-26 01:45:05    2009-01-06      NaN         2      4   
...           ...                 ...           ...      ...       ...    ...   
155316          0 2017-03-30 15:20:35    2017-02-17      NaN         0  22940   
155317          0 2017-03-30 15:20:35    2017-02-17      NaN         0  22940   
155318          0 2017-03-30 15:20:35    2017-02-17      NaN         0  22940   
155319          0 2017-03-30

In [ ]:
import os

# Check if the file exists at the specified path
if os.path.exists(tracks_csv_path):
    print("File found, proceeding with loading.")
    tracks = load(tracks_csv_path)
else:
    print(f"File not found at {tracks_csv_path}. Please check the file path.")


File found, proceeding with loading.


In [ ]:
# After confirming that tracks.csv is loaded successfully

# Explore the first few entries in the dataframe
print(tracks.head())

# Check for missing values in tracks
print(tracks.isnull().sum())

# Basic statistical analysis of numerical columns
print(tracks.describe())

# Load features.csv
features_csv_path = '/content/fma_metadata/features.csv'  # update with your actual path
if os.path.exists(features_csv_path):
    print("Loading features.csv")
    features = load(features_csv_path)
    print(features.info())

    # Handle missing values
    # If missing values are not numerous and are not critical, you can drop them
    features.dropna(inplace=True)
else:
    print(f"File not found at {features_csv_path}. Please check the file path.")


            album                                                          \
         comments        date_created date_released engineer favorites id   
track_id                                                                    
2               0 2008-11-26 01:44:45    2009-01-05      NaN         4  1   
3               0 2008-11-26 01:44:45    2009-01-05      NaN         4  1   
5               0 2008-11-26 01:44:45    2009-01-05      NaN         4  1   
10              0 2008-11-26 01:45:08    2008-02-06      NaN         4  6   
20              0 2008-11-26 01:45:05    2009-01-06      NaN         2  4   

                                                                           \
                                        information listens producer tags   
track_id                                                                    
2                                           <p></p>    6073      NaN   []   
3                                           <p></p>    6073      NaN   []  

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# Load the tracks DataFrame with the correct hierarchical column indexing
tracks = pd.read_csv(tracks_csv_path, index_col=0, header=[0, 1], low_memory=False)

# Access the 'genre_top' field directly
# The name 'genre_top' will be used assuming it is under the 'track' level of the DataFrame
genre_top_series = tracks[('track', 'genre_top')]

# Now we need to flatten the multi-level index for features
# Convert the multi-level column names into a single level by concatenating the levels
features.columns = ['_'.join(col).strip() for col in features.columns.values]

# Filter out tracks without genre information
genre_top_series = genre_top_series.dropna()

# Align the indices of the tracks and features DataFrame, we only keep rows that are present in both
common_indices = features.index.intersection(genre_top_series.index)
features_filtered = features.loc[common_indices]
genre_top_filtered = genre_top_series.loc[common_indices]

# Prepare the features and labels for machine learning
X = features_filtered
y = genre_top_filtered

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the feature values
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize the KNeighborsClassifier
knn_model = KNeighborsClassifier(n_neighbors=10, weights='distance')

# Train the classifier
knn_model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = knn_model.predict(X_test_scaled)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy of the KNN classifier: {accuracy:.2f}')


Accuracy of the KNN classifier: 0.60


In [ ]:
!pip install librosa pydub


In [ ]:
ls /content


features.py  fma_metadata/  fma_small/  __pycache__/  sample_data/  utils.py


In [ ]:
ls /content/fma_small

000/  011/  022/  033/  044/  055/  066/  077/  088/  099/  110/  121/  132/  143/  154/
001/  012/  023/  034/  045/  056/  067/  078/  089/  100/  111/  122/  133/  144/  155/
002/  013/  024/  035/  046/  057/  068/  079/  090/  101/  112/  123/  134/  145/  checksums
003/  014/  025/  036/  047/  058/  069/  080/  091/  102/  113/  124/  135/  146/  README.txt
004/  015/  026/  037/  048/  059/  070/  081/  092/  103/  114/  125/  136/  147/
005/  016/  027/  038/  049/  060/  071/  082/  093/  104/  115/  126/  137/  148/
006/  017/  028/  039/  050/  061/  072/  083/  094/  105/  116/  127/  138/  149/
007/  018/  029/  040/  051/  062/  073/  084/  095/  106/  117/  128/  139/  150/
008/  019/  030/  041/  052/  063/  074/  085/  096/  107/  118/  129/  140/  151/
009/  020/  031/  042/  053/  064/  075/  086/  097/  108/  119/  130/  141/  152/
010/  021/  032/  043/  054/  065/  076/  087/  098/  109/  120/  131/  142/  153/


In [ ]:
ls /content/fma_small/001

001039.mp3  001087.mp3  001270.mp3  001510.mp3  001673.mp3  001687.mp3  001720.mp3  001924.mp3
001040.mp3  001102.mp3  001276.mp3  001544.mp3  001680.mp3  001688.mp3  001732.mp3  001925.mp3
001066.mp3  001193.mp3  001277.mp3  001642.mp3  001681.mp3  001689.mp3  001733.mp3  001929.mp3
001069.mp3  001195.mp3  001278.mp3  001644.mp3  001682.mp3  001701.mp3  001735.mp3  001930.mp3
001073.mp3  001196.mp3  001417.mp3  001649.mp3  001683.mp3  001702.mp3  001736.mp3
001075.mp3  001197.mp3  001427.mp3  001661.mp3  001684.mp3  001703.mp3  001883.mp3
001082.mp3  001249.mp3  001443.mp3  001663.mp3  001685.mp3  001704.mp3  001891.mp3
001083.mp3  001259.mp3  001482.mp3  001666.mp3  001686.mp3  001706.mp3  001893.mp3


In [ ]:
import os
os.environ['AUDIO_DIR'] = '/content/fma_small'


In [ ]:
!pip install python-dotenv

In [ ]:
!python /content/features.py


**Train the Model**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the extracted features
features_path = '/content/features.csv'
features = pd.read_csv(features_path, index_col=0, header=[0, 1, 2])
features.columns = ['_'.join(col).strip() for col in features.columns.values]

# Load the tracks metadata which includes genres
tracks_path = '/content/fma_metadata/tracks.csv'
tracks = pd.read_csv(tracks_path, index_col=0, header=[0, 1])
tracks.columns = ['_'.join(col).strip() for col in tracks.columns.values]

# Extract genres as labels
y = tracks['track_genre_top']

# Align Data
common_indices = features.index.intersection(y.index)
features_filtered = features.loc[common_indices]
y_filtered = y.loc[common_indices]

# Drop columns where all values are NaN and replace any inf/-inf with NaN
features_filtered.replace([np.inf, -np.inf], np.nan, inplace=True)
features_filtered.dropna(axis=1, how='all', inplace=True)
features_filtered.fillna(0, inplace=True)

# Handle missing labels if any
y_filtered.fillna('Unknown', inplace=True)  # Assuming 'Unknown' is an acceptable category for your task

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(features_filtered, y_filtered, test_size=0.2, random_state=42)

# Check if any NaN values remain
if X_train.isna().any().any() or X_test.isna().any().any():
    print("NaN values still exist in the dataset!")
else:
    print("No NaN values present. Proceeding with scaling.")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize and train the KNeighborsClassifier
knn_model = KNeighborsClassifier(n_neighbors=10, weights='distance')
knn_model.fit(X_train_scaled, y_train)

# Make predictions and evaluate the model
y_pred = knn_model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy of the KNN classifier on the test set: {accuracy:.2f}')
print(classification_report(y_test, y_pred))


No NaN values present. Proceeding with scaling.
Accuracy of the KNN classifier on the test set: 0.53


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                     precision    recall  f1-score   support

              Blues       0.00      0.00      0.00        16
          Classical       0.00      0.00      0.00       254
            Country       0.00      0.00      0.00        44
     Easy Listening       0.00      0.00      0.00         3
         Electronic       0.00      0.00      0.00      1900
       Experimental       0.67      0.00      0.01      2140
               Folk       0.58      0.02      0.05       563
            Hip-Hop       0.56      0.03      0.05       746
       Instrumental       0.00      0.00      0.00       423
      International       0.30      0.05      0.09       267
               Jazz       0.00      0.00      0.00       109
Old-Time / Historic       0.00      0.00      0.00       122
                Pop       0.17      0.00      0.00       444
               Rock       0.45      0.00      0.01      2886
           Soul-RnB       0.00      0.00      0.00        45
             Spoken    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
